In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"priyanshu1232","key":"4b5088cd39b5bd5e0ad3af057a47cc40"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d hariharasudhanas/twitter-emoji-prediction

Dataset URL: https://www.kaggle.com/datasets/hariharasudhanas/twitter-emoji-prediction
License(s): other
100% 3.50M/3.50M [00:01<00:00, 3.92MB/s]
100% 3.50M/3.50M [00:01<00:00, 2.90MB/s]


In [7]:
!unzip /content/twitter-emoji-prediction.zip

Archive:  /content/twitter-emoji-prediction.zip
  inflating: Mapping.csv             
  inflating: OutputFormat.csv        
  inflating: Test.csv                
  inflating: Train.csv               


In [8]:
! pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 11.9 MB/s eta 0:00:00


In [9]:
import emoji
import numpy as np
import pandas as pd

In [10]:
mappings = pd.read_csv("/content/Mapping.csv")
test = pd.read_csv("/content/Test.csv")
train = pd.read_csv("/content/Train.csv")
ouptut = pd.read_csv("/content/OutputFormat.csv")

In [11]:
mappings

,Unnamed: 0,emoticons,number
0,0,😜,0
1,1,📸,1
2,2,😍,2
3,3,😂,3
4,4,😉,4
5,5,🎄,5
6,6,📷,6
7,7,🔥,7
8,8,😘,8
9,9,❤,9


In [12]:
mappings = mappings.drop("Unnamed: 0" , axis =1)

In [13]:
mappings.head()

,emoticons,number
0,😜,0
1,📸,1
2,😍,2
3,😂,3
4,😉,4


In [14]:
mappings = mappings.set_index('number')

In [15]:
mappings.head()

,emoticons
number,
0,😜
1,📸
2,😍
3,😂
4,😉


In [16]:
mapping_dict = mappings.to_dict()
mapping_dict = mapping_dict['emoticons']

In [17]:
mapping_dict

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [18]:
train.head()

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3


In [19]:
train['TEXT'][0]

'Vacation wasted ! #vacation2017 #photobomb #tired #vacationwasted #mcgar30 #miami @ Port of…\n'

In [20]:
train = train.drop("Unnamed: 0" ,axis = 1)

In [21]:
train.head()

,TEXT,Label
0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,Been friends since 7th grade. Look at us now w...,2
3,This is what it looks like when someone loves ...,3
4,RT @user this white family was invited to a Bl...,3


In [22]:
import string
import re

def data_clean(tweet):
    tweet = str(tweet).lower()
    rm_mention = re.sub(r'@[A-Za-z0-9]+', '', tweet)                       # remove @mentions
    rm_rt = re.sub(r'RT[/s]+', '', rm_mention)                             # remove RT
    rm_links = re.sub(r'http\S+', '', rm_rt)                               # remove hyperlinks
    rm_links = re.sub(r'https?:\/\/\S+','', rm_links)
    rm_nums = re.sub('[0-9]+', '', rm_links)                               # remove numbers
    rm_punc = [char for char in rm_nums if char not in string.punctuation] # remove punctuations
    rm_punc = ''.join(rm_punc)
    cleaned = rm_punc

    return cleaned


In [23]:
train["TEXT"] = train["TEXT"].apply(data_clean)

In [24]:
train['TEXT'][0]

'vacation wasted  vacation photobomb tired vacationwasted mcgar miami  port of…\n'

In [25]:
x_train = train['TEXT'].values
y_train = train['Label'].values

In [26]:
print("X_train shape : ",x_train.shape,"\nY_train shape : ",y_train.shape)

X_train shape :  (70000,) 
Y_train shape :  (70000,)


In [27]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-07-02 02:52:51--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-07-02 02:52:51--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-07-02 02:52:52--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [28]:
!unzip /content/glove.6B.zip

Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [29]:
file = open("/content/glove.6B.50d.txt", encoding = 'utf8')

In [30]:
def initialize_embedding_matrix(file):
  embedding_matrix = {}
  for line in file:
    values = line.split()
    word = values[0]
    embedding = np.array(values[1:],dtype = 'float64')
    embedding_matrix[word] = embedding

  return embedding_matrix

In [36]:
embedding_matrix = initialize_embedding_matrix(file)

In [38]:
def get_embedding_data(data, max_len):
  embedding_data = np.zeros((len(data),max_len,50))
  for idx in range(data.shape[0]):
    word_in_sentence = data[idx].split()
    for i in range(len(word_in_sentence)):
      if (embedding_matrix.get(word_in_sentence[i].lower())) is not None and i < max_len:
         embedding_data[idx][i] = embedding_matrix[word_in_sentence[i].lower()]

  return embedding_data

In [40]:
embedding_data = get_embedding_data(x_train,20)

In [41]:
embedding_data.shape

(70000, 20, 50)

In [42]:
from keras.utils import to_categorical

In [43]:
y_train = to_categorical(y_train)

In [44]:
y_train[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [45]:
embedding_data.shape

(70000, 20, 50)

In [46]:
y_train.shape

(70000, 20)

---------------------------------

In [59]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional

In [60]:
model = Sequential()

In [61]:
model.add(Bidirectional(LSTM(units = 512, return_sequences= True , input_shape = (20,50)))) # (168,50)
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units = 128)))
model.add(Dropout(0.3))
model.add(Dense(units = 128,activation = 'relu'))
model.add(Dense(units = 64,activation= 'relu'))
model.add(Dense(units = 32,activation = 'relu'))
model.add(Dense(units = 20,activation = 'relu'))
model.add(Dense(units = 20,activation='softmax'))

In [62]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy' ,metrics = ['accuracy'])

In [63]:
model.fit(embedding_data,y_train,validation_split = 0.2 ,shuffle = True , batch_size = 64, epochs = 10)

Epoch 1/10
875/875 [==============================] - 38s 28ms/step - loss: 2.8649 - accuracy: 0.2148 - val_loss: 2.7826 - val_accuracy: 0.2183
Epoch 2/10
875/875 [==============================] - 16s 19ms/step - loss: 2.7533 - accuracy: 0.2149 - val_loss: 2.7395 - val_accuracy: 0.2183
Epoch 3/10
875/875 [==============================] - 19s 21ms/step - loss: 2.7361 - accuracy: 0.2149 - val_loss: 2.7356 - val_accuracy: 0.2183
Epoch 4/10
875/875 [==============================] - 17s 19ms/step - loss: 2.7350 - accuracy: 0.2149 - val_loss: 2.7356 - val_accuracy: 0.2183
Epoch 5/10
875/875 [==============================] - 17s 20ms/step - loss: 2.7350 - accuracy: 0.2149 - val_loss: 2.7355 - val_accuracy: 0.2183
Epoch 6/10
875/875 [==============================] - 19s 22ms/step - loss: 2.7350 - accuracy: 0.2149 - val_loss: 2.7356 - val_accuracy: 0.2183
Epoch 7/10
875/875 [==============================] - 18s 20ms/step - loss: 2.7350 - accuracy: 0.2149 - val_loss: 2.7355 - val_accuracy:

In [52]:
mapping_dict

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [64]:
model.save("Emoji_Pred.h5")

In [65]:
from tensorflow.keras.models import load_model
loaded_model = load_model("Emoji_Pred.h5")

In [68]:
text = input(" ")

test_data = np.array([text])
test_embedding = get_embedding_data(test_data,20)


y_pred = loaded_model.predict(test_embedding)[0]
label = np.argmax(y_pred)
emoji_got = mapping_dict[label]


print(text +" {}".format(emoji_got))

 i just love making ML projects 
1/1 [==============================] - 0s 43ms/step
i just love making ML projects  ❤
